# Recognizing handwritten digits using a neural network

We have now reached the point where we can tackle a very interesting task: applying the knowledge we have gained with machine learning in general, and `Flux.jl` in particular, to create a neural network that can recognize handwritten digits! The data are from a data set called MNIST, which has become a classic in the machine learning world.

[We could also try to apply the techniques to the original images of fruit instead. However, the fruit images are much larger than the MNIST images, which makes the learning a suitable neural network too slow.]

## Data Munging


As we know, the first difficulty with any new data set is locating it, understanding what format it is stored in, reading it in and decoding it into a useful data structure in Julia.

The original MNIST data is available [here](http://yann.lecun.com/exdb/mnist); see also the [Wikipedia page](https://en.wikipedia.org/wiki/MNIST_database). However, the format that the data is stored in is rather obscure.

Fortunately, various packages in Julia provide nicer interfaces to access it. We will use the one provided by `Flux.jl`.

The data are images of handwritten digits, and the corresponding labels that were determined by hand (i.e. by humans). Our job will be to get the computer to **learn** to recognize digits by learning, as usual, the function that relates the input and output data.

### Loading and examining the data

First we load the required packages:

In [1]:
using Flux, Images, MLDatasets, OneHotArrays, MLUtils

Now we read the data:

In [2]:
xtrain, ytrain = MLDatasets.MNIST(:train)[:]
xtest, ytest = MLDatasets.MNIST(:test)[:]

(features = [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0;;; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0;;; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0;;; … ;;; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0;;; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0;;; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], targets = [7, 2, 1, 0, 4, 1, 4, 9, 5, 9  …  7, 8, 9, 0, 1, 2, 3, 4, 5, 6])

In [3]:
size(xtrain)

(28, 28, 60000)

Looking at this we see that there are 60000 images of size 28x28

#### Creating the features

We want to create a vector of feature vectors, each with the floating point values of the 784 pixels.


In [4]:
xtrain = Flux.flatten(xtrain)
xtest = Flux.flatten(xtest) #add semicolon to suppress the output

784×10000 Matrix{Float32}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0

#### Getting number of inputs and outputs

In [5]:
n_inputs = size(xtrain)[1] # 784
n_outputs = length(unique(ytrain)) # 10

10

#### Creating the labels

We can just use `onehotbatch` to create them:

In [6]:
ytrain

60000-element Vector{Int64}:
 5
 0
 4
 1
 9
 2
 1
 3
 1
 4
 3
 5
 3
 ⋮
 7
 8
 9
 2
 9
 5
 1
 8
 3
 5
 6
 8

In [7]:
ytrain, ytest = onehotbatch(ytrain, 0:9), onehotbatch(ytest, 0:9)

(Bool[0 1 … 0 0; 0 0 … 0 0; … ; 0 0 … 0 1; 0 0 … 0 0], Bool[0 0 … 0 0; 0 0 … 0 0; … ; 0 0 … 0 0; 0 0 … 0 0])

In [8]:
ytrain

10×60000 OneHotMatrix(::Vector{UInt32}) with eltype Bool:
 ⋅  1  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  …  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅
 ⋅  ⋅  ⋅  1  ⋅  ⋅  1  ⋅  1  ⋅  ⋅  ⋅  ⋅     ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  1  ⋅  ⋅  ⋅  ⋅  ⋅
 ⋅  ⋅  ⋅  ⋅  ⋅  1  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅     ⋅  ⋅  ⋅  1  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅
 ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  1  ⋅  ⋅  1  ⋅  1     ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  1  ⋅  ⋅  ⋅
 ⋅  ⋅  1  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  1  ⋅  ⋅  ⋅     ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅
 1  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  1  ⋅  …  ⋅  ⋅  ⋅  ⋅  ⋅  1  ⋅  ⋅  ⋅  1  ⋅  ⋅
 ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅     ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  1  ⋅
 ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅     1  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅
 ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅     ⋅  1  ⋅  ⋅  ⋅  ⋅  ⋅  1  ⋅  ⋅  ⋅  1
 ⋅  ⋅  ⋅  ⋅  1  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅     ⋅  ⋅  1  ⋅  1  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅

#### Dataloader

In [9]:
function trainloader(size)
    train_loader = DataLoader((xtrain, ytrain), batchsize = size, shuffle = true)
    return train_loader
end


trainloader (generic function with 1 method)

### Setting up a neural network

In the previous notebooks, we arranged the input data for Flux as a `Vector` of `Vector`s.
Now we will use an alternative arrangement, as a matrix, since that allows `Flux` to use matrix operations, which are more efficient.

The column $i$ of the matrix is a vector consisting of the $i$th data point $\mathbf{x}^{(i)}$.  Similarly, the desired outputs are given as a matrix, with the $i$th column being the desired output $\mathbf{y}^{(i)}$.

Now we must set up a neural network. Since the data is complicated, we may expect to need several layers.
But we can start with a single layer.

- The network will take as inputs the vectors $\mathbf{x}^{(i)}$, so the input layer has $n$ nodes.

- The output will be a one-hot vector encoding the digit from 1 to 9 or 0 that is desired. There are 10 possible categories, so we need an output layer of size 10.

It is then our task as neural network designers to insert layers between these input and output layers, whose weights will be tuned during the learning process. *This is an art, not a science*! But major advances have come from finding a good structure for the network.

In [10]:
n_inputs

784

In [11]:
model = Chain(Dense(n_inputs, n_outputs, identity), softmax)

Chain(
  Dense(784 => 10),                     # 7_850 parameters
  NNlib.softmax,
) 

In [12]:
model(xtrain)

10×60000 Matrix{Float32}:
 0.0584831  0.0791119  0.0740139  …  0.0960662  0.113541   0.0710475
 0.0986182  0.0318864  0.110993      0.0720082  0.0755236  0.0503368
 0.150523   0.18543    0.0926961     0.058545   0.0853627  0.157223
 0.0907781  0.060158   0.131178      0.053839   0.0396145  0.073929
 0.163655   0.200239   0.102506      0.121097   0.166219   0.0668063
 0.179576   0.104816   0.107718   …  0.250003   0.114821   0.275188
 0.0431743  0.0553528  0.0644514     0.0603587  0.052873   0.0510857
 0.119632   0.121493   0.13477       0.0976854  0.125548   0.0897675
 0.0501339  0.113836   0.127531      0.070316   0.17275    0.0794764
 0.0454265  0.0476765  0.0541426     0.120081   0.0537472  0.0851392

In [13]:
L(x,y) = Flux.crossentropy(model(x), y)
ps = Flux.params(model)
η = 0.1
opt = Descent(η)

Descent(0.1)

In [14]:
trainbatch = trainloader(60000)

DataLoader{Tuple{Matrix{Float32}, OneHotMatrix{UInt32, 10, Vector{UInt32}}}, Random._GLOBAL_RNG, Val{nothing}}((Float32[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], Bool[0 1 … 0 0; 0 0 … 0 0; … ; 0 0 … 0 1; 0 0 … 0 0]), 60000, false, true, true, false, Val{nothing}(), Random._GLOBAL_RNG())

In [15]:
L(xtrain, ytrain)

2.3590982f0

### Training

In [16]:
Flux.train!(L, ps, trainbatch, opt)

In [17]:
@time Flux.train!(L, ps, trainbatch, opt)

  1.025552 seconds (149 allocations: 382.960 MiB, 30.70% gc time)


In [18]:
for i in 1:5
    for i in 1:100
        Flux.train!(L, ps, trainbatch, opt)
    end
    println(L(xtrain, ytrain))
end

0.61136353
0.49229488
0.44284844
0.414302
0.3951605


### Testing Phase

In [19]:
ytrain, ytest = onecold(ytrain, 0:9), onecold(ytest, 0:9)

([5, 0, 4, 1, 9, 2, 1, 3, 1, 4  …  9, 2, 9, 5, 1, 8, 3, 5, 6, 8], [7, 2, 1, 0, 4, 1, 4, 9, 5, 9  …  7, 8, 9, 0, 1, 2, 3, 4, 5, 6])

In [55]:
model(xtrain)[11:20]

10-element Vector{Float32}:
 0.9962936
 4.6798272f-7
 0.00013244376
 0.0002770524
 2.3846585f-6
 0.0027600925
 0.00018166976
 9.3040995f-5
 0.00020297637
 5.6219604f-5

In [56]:
ytrain[2]

0

In [29]:
predictiontrain(n) = onecold(model(xtrain), 0:9)[n]
predictiontest(n) = onecold(model(xtest), 0:9)[n]

predictiontest (generic function with 1 method)

In [57]:
predictiontrain(2)

0

### Evaluate
Here we evaluate the training and test accuracy (this takes a while)

In [44]:
trainingaccuracy = sum(predictiontrain(i) == ytrain[i] for i in 1:60000)/60000

0.8938

In [45]:
testaccuracy = sum(predictiontest(i) == ytest[i] for i in 1:10000)/10000

0.8999

## Improving the prediction

So far we have used a single layer. In order to improve the prediction, we probably need to use more layers. Try adding more layers yourself and see how the performance changes.

In [46]:
n_hidden = 32
model2 = Chain(
    Dense(n_inputs, n_hidden, relu),
    Dense(n_hidden, n_outputs, identity),
    softmax)


Chain(
  Dense(784 => 32, relu),               # 25_120 parameters
  Dense(32 => 10),                      # 330 parameters
  NNlib.softmax,
)                   # Total: 4 arrays, 25_450 parameters, 99.664 KiB.

In [47]:
L2(x, y) = Flux.crossentropy(model2(x), y)
ps2 = Flux.params(model2)
opt2 = Adam(0.001)

Adam(0.001, (0.9, 0.999), 1.0e-8, IdDict{Any, Any}())

In [48]:
ytrain, ytest = onehotbatch(ytrain, 0:9), onehotbatch(ytest, 0:9)

(Bool[0 1 … 0 0; 0 0 … 0 0; … ; 0 0 … 0 1; 0 0 … 0 0], Bool[0 0 … 0 0; 0 0 … 0 0; … ; 0 0 … 0 0; 0 0 … 0 0])

In [49]:
trainbatch2 = trainloader(60000)

DataLoader{Tuple{Matrix{Float32}, OneHotMatrix{UInt32, 10, Vector{UInt32}}}, Random._GLOBAL_RNG, Val{nothing}}((Float32[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], Bool[0 1 … 0 0; 0 0 … 0 0; … ; 0 0 … 0 1; 0 0 … 0 0]), 60000, false, true, true, false, Val{nothing}(), Random._GLOBAL_RNG())

In [50]:
for i in 1:5
    for i in 1:100
        Flux.train!(L2, ps2, trainbatch2, opt2)
    end
    println(L2(xtrain, ytrain))
end

0.38160208
0.28114498
0.23610607
0.20168594
0.17544319


In [51]:
ytrain, ytest = onecold(ytrain, 0:9), onecold(ytest, 0:9)

([5, 0, 4, 1, 9, 2, 1, 3, 1, 4  …  9, 2, 9, 5, 1, 8, 3, 5, 6, 8], [7, 2, 1, 0, 4, 1, 4, 9, 5, 9  …  7, 8, 9, 0, 1, 2, 3, 4, 5, 6])

In [52]:
predictiontrain2(n) = onecold(model2(xtrain), 0:9)[n]
predictiontest2(n) = onecold(model2(xtest), 0:9)[n]

predictiontest2 (generic function with 1 method)

In [53]:
trainingaccuracy = sum(predictiontrain2(i) == ytrain[i] for i in 1:60000)/60000

0.9513333333333334

In [54]:
testaccuracy = sum(predictiontest2(i) == ytest[i] for i in 1:10000)/10000

0.9462

We can also try different methods like CNNs